In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from oracle import Oracle
from accuracy_test import accuracy_check, accuracy_check_iter, accuracy_check_err
import time

In [ ]:
def decentralizedSGD(x0, lr, max_iter, eps, W, oracle):
    
    x_t = x0
    x_prev = x0
    
    itr = 0
    error = np.inf
    current_sum = 0.
    
    errors = []
    
    x_star, _ = oracle.getMin()
    
    while (itr < max_iter) and (error > eps):
         
        noise = np.random.normal(0, 1, size=x_t.shape) / 100
        
        f, df = oracle(x_t + noise)
        
        x_prev = x_t
        
        y_t = x_t - lr * df
        x_t = y_t @ W
        
        if (oracle.getType() == "strongly convex") and (itr > 0):
            err, sum_ = accuracy_check_iter(x_prev, current_sum, itr, oracle, x_t)
            error = err
            current_sum = sum_
            errors.append(error)
        elif (oracle.getType() != "strongly convex"):
            err, sum_ = accuracy_check_iter(x_t, current_sum, itr, oracle)
            error = err
            current_sum = sum_
            errors.append(error)
        #error = accuracy_check_err(x_t, x_star)
        #errors.append(error)
        
        itr += 1
        
    return x_t, itr, errors

def MetropolisHastings(W):
    
    degrees = np.sum(W, axis=1)
    
    for i in range(W.shape[0]):
        for j in range(i+1, W.shape[0]):
            
            if W[i, j] != 0:
                
                weight = 1 / (1 + max(degrees[i], degrees[j]))
                
                W[i, j] = weight
                W[j, i] = weight
    
    for i in range(W.shape[0]):
        W[i, i] = 1 - (np.sum(W, axis = 0)[i] - 1.0) # Remove 1 because counts self weight of 1 from adjacency matrix W
                
    return W


def toMatrix(n_nodes, adj_list):
    
    matrix = np.zeros((n_nodes, n_nodes))
    
    for node in adj_list:
        
        neighbors = adj_list[node]
        
        for neighbor in neighbors:
            
            matrix[node, neighbor] = 1
            matrix[neighbor, node] = 1
            
    return matrix.astype(np.float64)


def buildTopology(n_nodes, topology):
    
    if topology == "dense":
        
        W = np.ones((n_nodes, n_nodes))

        return MetropolisHastings(W.astype(np.float64))
    
    elif topology == "ring":
        
        connections = {} # adjacency list
        
        for node in range(n_nodes):
            
            if node == n_nodes - 1:
                connections[node] = [node, 0]
            else:
                connections[node] = [node, node + 1]
        
        W = toMatrix(n_nodes, connections)
        
        return MetropolisHastings(W)
    
    elif topology == "centralized":
        
        connections = {} # adjacency list
        
        connections[0] = [0]
        for node in range(1, n_nodes):
            connections[node] = [node, 0] # node 0 is central node
            
        W = toMatrix(n_nodes, connections)
        
        return MetropolisHastings(W)
    
    else:
        print("Wrong topology")
        

def experiment(list_nodes, n_params, topology, func_type, max_iter, lr, error_threshold, number_exp = 10):
    
    means = []
    results = []
    
    oracle = Oracle(func_type, n_params)
    
    for n_nodes in list_nodes:
        start = time.time()
        
        print("Starting experiment on {} nodes.".format(n_nodes))
        
        temp = []
        
        for _ in range(number_exp): # Do average
            
            x0 = np.ones((n_params, n_nodes)).astype(np.float64) #np.random.randn(n_params, n_nodes)
            W = buildTopology(n_nodes, topology)
            
            x_t, itr, errors = decentralizedSGD(x0, lr, max_iter, error_threshold, W, oracle)

            temp.append(itr)
        
        mean = np.mean(temp)
        
        print(f'DEBUG num.nides {n_nodes} time {time.time() - start}')
        print("Converged within {} in {} iterations on average.".format(error_threshold, mean))
        print()
        
        means.append(mean)
        results.append(temp)
    
    return np.array(means), np.array(results)
    

def plotIter(num_nodes, iters):
    
    plt.plot(num_nodes, iters)
    plt.xlabel("Number of nodes")
    plt.ylabel("Number of iterations")
    plt.show()

# Experiments

In [ ]:
save_folder = "data/"

n_params = 8
max_iter = np.inf
number_exp = 5

num_nodes = list(range(2, 1004, 200))

## Non convex function

In [ ]:
func_type = "non convex"

In [ ]:
topology = "dense"
lr = 1e-1
threshold = 1e-3

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)

In [ ]:
topology = "ring"
lr = 1e-1
threshold = 1e-3

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)

In [ ]:
topology = "centralized"
lr = 1e-1
threshold = 1e-3

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)

## Convex function

In [ ]:
func_type = "convex"

In [ ]:
topology = "dense"
lr = 1e-1
threshold = 1e-2

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)

In [ ]:
topology = "ring"
lr = 1e-1
threshold = 1e-2

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)

In [ ]:
topology = "centralized"
lr = 1e-1
threshold = 1e-2

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)

## Strongly convex function

In [ ]:
func_type = "convex"

In [ ]:
topology = "dense"
lr = 1e-1
threshold = 1e-2

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)

In [ ]:
topology = "ring"
lr = 1e-1
threshold = 1e-2

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)

In [ ]:
topology = "centralized"
lr = 1e-1
threshold = 1e-2

means, results = experiment(num_nodes, n_params, topology, func_type, max_iter, lr, threshold, number_exp)

plotIter(num_nodes, means)

filename = save_folder + func_type.replace(" ", "") + "_" + topology + ".npy"
print(filename)

with open(filename, 'wb') as f:
    np.save(f, results)